# Activate Django

In [12]:
import os; os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'dplback.settings')
import django; django.setup()

# Imports

In [13]:
from cryptography.hazmat.primitives.asymmetric import ed25519
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization
import jwt
from cryptography.fernet import Fernet
from dplapp.models import AppSettingsModel, TokensModel
from django.utils import timezone
from datetime import timezone as builtin_tz
from argon2 import PasswordHasher
from django.conf import settings
from dplapp.serializers import TicketSerializer, MainRequestSerializer
from dplapp.utils import *
from datetime import datetime, timedelta
from datetime import timezone as builtin_tz
from django.utils import timezone
import uuid
from django.core.exceptions import ValidationError
import re
import hashlib
from time import sleep
from typing import Any
import requests
import json
import pickle

# Token client class definition (mimics mobile app)

In [ ]:
DEV_URL = "http://127.0.0.1:8000"

In [ ]:
class TokenClient:

    app_version = "v0.0.5"
    api_url = "/mainrequest/"
    base_url = DEV_URL


    def __init__(self, pin = "1234567890", saved_token = None):

        if not saved_token:
            self.private_key, self.public_key = self._generate_keypair()
            self.private_bytes, self.public_bytes = self._get_key_bytes()
            self.ticket = ""
        else:
            decoded_data = self._restore_saved_token(saved_token)
            if decoded_data:
                self.private_key, self.public_key, self.ticket = decoded_data

        self.fingerprint = self._get_fingerprint()
        self.pin = self._get_pin(pin)

    def _get_url(self):
        return self.base_url + self.api_url

    def _generate_keypair(self):
        private_key =  rsa.generate_private_key(key_size=2048, public_exponent=65537)
        public_key = private_key.public_key()
        return (private_key, public_key)
    
    def _get_key_bytes(self):
        private_bytes = self.private_key.private_bytes(
                encoding=serialization.Encoding.PEM,
                format=serialization.PrivateFormat.PKCS8,
                encryption_algorithm=serialization.NoEncryption(),
            ).decode()
        private_bytes = re.sub(r'(\r\n)|\n', '', private_bytes)

        public_bytes = self.public_key.public_bytes(
                    encoding=serialization.Encoding.PEM,
                    format=serialization.PublicFormat.SubjectPublicKeyInfo,
                ).decode()
        public_bytes = re.sub(r'(\r\n)|\n', '', public_bytes)

        return (private_bytes, public_bytes)
    
    def _get_fingerprint(self):
        return ' '.join(f'{byte:02X}' for byte in hashlib.sha256(self.public_bytes.encode('utf-8')).digest()[:6])
    
    def _get_pin(self, pin):
        return hashlib.sha256(f"{pin}{self.fingerprint}".encode('utf-8')).hexdigest()
    
    def _get_jwt_dict(self):
        jwt_dict = {
            "version": self.app_version,
            "request_time": timezone.now().isoformat(),
            "public_key": self.public_bytes,
            "pin": self.pin,
            "ticket": self.ticket
        }
        return jwt_dict
    
    def _encode_jwt_token(self, jwt_dict : dict[str, Any]):
        return jwt.encode(jwt_dict, self.private_key, algorithm="RS256")
    
    def _get_request_data(self, jwt_token):
        return {'token': jwt_token}
    
    def _get_ticket_from_response(self, response):
        return json.loads(response)['ticket']

    def send_mainrequest(self):
        jwt_dict = self._get_jwt_dict()
        jwt_token = self._encode_jwt_token(jwt_dict)
        request_data = self._get_request_data(jwt_token)

        response = requests.post(self._get_url(), json=request_data)

        if response.status_code == 200:
            self.ticket = self._get_ticket_from_response(response.content)

        return (response.status_code, response.content)

    def save_token(self):
        pickled_data = pickle.dumps((self.private_key, self.public_key, self.ticket))
        base64_bytes = base64.b64encode(pickled_data)
        return base64_bytes.decode()


    def _restore_saved_token(self, encoded_string : str):
        base64_bytes = encoded_string.encode()
        pickled_data = base64.b64decode(base64_bytes)
        unpickled_data = pickle.loads(pickled_data)
        if (type(unpickled_data) != type): return None
        if len(unpickled_data) != 3: return None
        return unpickled_data
    
    def __str__(self):
        return self.fingerprint
    
    def __repr__(self):
        return f"Token({self.fingerprint})"

# Code playground

In [10]:
class MassTokenCreator:

    tokens_dict : dict[TokenClient] = dict()

    def __init__(self, amount):
        for _ in range(amount):
            token = TokenClient()
            token.send_mainrequest()
            self.tokens_dict[token.fingerprint] = token
        print(list(self.tokens_dict.keys()))
        
    @classmethod
    def send_requests(self, tokens: list[str] | None = None):
        if tokens:
            for fingerprint in tokens:
                print(fingerprint, self.tokens_dict[fingerprint].send_mainrequest())
        else:
            for token in self.tokens_dict.values():
                print(token.fingerprint, token.send_mainrequest())

    @classmethod
    def clear_tokens(self, tokens: list[str] | None = None):
        if tokens:
            for token in tokens:
                self.tokens_dict.pop(token)
        else:
            self.tokens_dict.clear()


In [11]:
MassTokenCreator.clear_tokens()
MassTokenCreator(2)

DEBUG 2025-04-01 17:35:13,810 connectionpool 19228 25908 Starting new HTTP connection (1): 127.0.0.1:8000


ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /mainrequest/ (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001BA91C67DD0>: Failed to establish a new connection: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение'))

In [ ]:
MassTokenCreator(2)

DEBUG 2025-04-01 16:50:45,195 connectionpool 19228 25908 Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG 2025-04-01 16:50:45,257 connectionpool 19228 25908 http://127.0.0.1:8000 "POST /mainrequest/ HTTP/1.1" 200 1460
DEBUG 2025-04-01 16:50:45,283 connectionpool 19228 25908 Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG 2025-04-01 16:50:45,344 connectionpool 19228 25908 http://127.0.0.1:8000 "POST /mainrequest/ HTTP/1.1" 200 1460


['FE 80 1B 67 13 AE', 'AC 38 EE C1 1C FF', '62 DC B3 25 9C FC', '60 1E 3B 62 C1 9B']


In [ ]:
MassTokenCreator(2)

In [ ]:
MassTokenCreator.send_requests()

DEBUG 2025-04-01 16:52:54,611 connectionpool 19228 25908 Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG 2025-04-01 16:52:54,679 connectionpool 19228 25908 http://127.0.0.1:8000 "POST /mainrequest/ HTTP/1.1" 200 1622
DEBUG 2025-04-01 16:52:54,681 connectionpool 19228 25908 Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG 2025-04-01 16:52:54,741 connectionpool 19228 25908 http://127.0.0.1:8000 "POST /mainrequest/ HTTP/1.1" 200 1622
DEBUG 2025-04-01 16:52:54,744 connectionpool 19228 25908 Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG 2025-04-01 16:52:54,805 connectionpool 19228 25908 http://127.0.0.1:8000 "POST /mainrequest/ HTTP/1.1" 200 1622
DEBUG 2025-04-01 16:52:54,807 connectionpool 19228 25908 Starting new HTTP connection (1): 127.0.0.1:8000
DEBUG 2025-04-01 16:52:54,866 connectionpool 19228 25908 http://127.0.0.1:8000 "POST /mainrequest/ HTTP/1.1" 200 1622


FE 80 1B 67 13 AE (200, b'{"ticket":"eyJhbGciOiJFZERTQSIsInR5cCI6IkpXVCJ9.eyJwYXlsb2FkIjoiZ0FBQUFBQm42LS0ycFFHUlFhaHRpdGJxOURuNnFnSnZrUzZzVVNVMm9XWXc3V1RYRnpETjlwTDVtWGJlVkpDYXdDblJyQ0JjMGhITjBMaVZibURFVjk1SDNtY0ZaLU9VSm1lR1NDTFdrWnptMHRYWHIxZ0lKM3FOR1dTdUhWaHFILXdSb2pMY0ZFa2NGaGQyMFZINkViOWJoQjd3OTh3cnNzdVZ0OHNDTHpTaEhEQzRhTDRlUkxrSmMyaUVqWFhoLWMzVnpYNXZuTS16S3Y1Y1FrRFlSb1ZNRnpRMnoyMnMwd3BUbGNyLUtwUGwyaVZocHpfQ04xOEdBV3JUNkN0ZVM3bEpEQ1p6RXVvdE9xTVE0bFdHM3FVaDJUNmNIU2FuUkZCZlpLbi1UN2xxeDdDdVo1SE5JcWVPUTVpeVNCc2sxSTZxbm9INGhQN2VFSlJSSWRNTHpRckJES1BRbGpEQWs2bWlVeDNZLTVPdHVyX3J2MnVfQW5TX3QzZ1V4ME81RU5GYWpyWHVRdlBpTG5BMWFXX0NYUjlXQ3FPajMwdTBtMGdGS3ZISzdOZjR6OWdzZWwyY1lwTTRCYWxRYUhRaURMcHJ2dnJZZ2FmLUY1Mkd4T0N5R1pQSlROX2Z0MzRqeDcxcWRleWNrWG1UU2IyRWtQQXlDVV9URTRfdXpsM0VSaWE1eE5IbmhIQTZqQmFydTV5ZzFaVEFzT2RuVzNvaGdFaUNNN1BGZFJIM1JMUmFTMVRncE4zWDVXSGtvQXlCYkNEM1BuYWNFaVdMU202ZXZjOUdXaU9DRDBGYTFPMkxXYVh1SFItSkJSV0NIdTVpa2lSWEpPQVBGelp0Q2FkMHlHMWlQdVcyckoxbTYyY3gyTDdkZTQ5cmZrUjAwZEswaG5tTHU1TjdMS1